<a href="https://colab.research.google.com/github/Danila-Kovalenko/My_Machine_Learning/blob/main/Die_Kunden_von_Beta_Bank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Die Kunden von Beta-Bank
Kunden begannen, die Beta Bank zu verlassen. Jeden Monat. Ein wenig, aber spürbar. Bankvermarkter haben berechnet, dass es günstiger ist, bestehende Kunden zu halten, als neue zu gewinnen.
Es muss vorhergesagt werden, ob der Kunde die Bank in naher Zukunft verlassen wird oder nicht. Ihnen werden historische Daten zum Kundenverhalten und zur Beendigung von Verträgen mit der Bank angezeigt.

Schritt 0: Wir importieren die nötige Bibliotheken

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

Schritt 1: Entfernen unnötiger Parameter

In [ ]:
data = pd.read_csv('/content/Churn.csv')
data = data.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

Schritt 2: Lücken im „Tenure“-Parameter schließen

In [ ]:
imputer = SimpleImputer(strategy='mean')
data['Tenure'] = imputer.fit_transform(data[['Tenure']])

Schritt 3: Kategoriale Merkmale kodieren

In [ ]:
categorical_features = ['Geography', 'Gender']
numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)])

Schritt 4: Teilen Sie die Daten in Features und Zielvariable auf

In [ ]:
X = data.drop(columns='Exited')
y = data['Exited']

Schritt 5: Teilen Sie die Daten in Trainings- und Testsätze auf

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Schritt 6: Einrichten des Modells mit hyperparametrischer Optimierung

In [ ]:
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__class_weight': [class_weight_dict]
}

model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', RandomForestClassifier(random_state=42))])

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['CreditScore',
                                                                          'Age',
                                                                          'Tenure',
                                                                          'Balance',
                                                                          'NumOfProducts',
                                                                          'HasCrCard',
                                                                          'IsActiveMember',
                                                                          'EstimatedSalary']),
                                                                        ('cat',
                                                                         OneHotEncoder(drop='first'),
                                                                         ['Geography',
                                                                          'Gender'])])),
                                       ('classifier',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'classifier__class_weight': [{0: 0.6293266205160478,
                                                       1: 2.4330900243309004}],
                         'classifier__max_depth': [None, 10, 20, 30],
                         'classifier__min_samples_leaf': [1, 2, 4],
                         'classifier__min_samples_split': [2, 5, 10],
                         'classifier__n_estimators': [100, 200, 300]},
             scoring='f1')

In [ ]:
# Bestes Modell nach der Optimierung
best_model = grid_search.best_estimator_

# Prognosen und Modellbewertung an einem Testmuster
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"Bestes F1-Maß: {f1}")
print(f"AUC-ROC: {roc auc}")
print(f'Beste Parameter: {grid_search.best_params_}')

Лучшая F1-мера: 0.6261467889908257
AUC-ROC: 0.8688672807105049
Лучшие параметры: {'classifier__class_weight': {0: 0.6293266205160478, 1: 2.4330900243309004}, 'classifier__max_depth': 10, 'classifier__min_samples_leaf': 4, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 200}
